In [1]:
%load_ext autoreload
import f3dasm
from f3dasm_simulate import openFoam
from PyFoam import FoamInformation
from pathlib import Path

2023-06-23 16:58:41,312 - Imported f3dasm (version: 1.1.0)
2023-06-23 16:58:41.773041: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-23 16:58:41.774261: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-23 16:58:41.797615: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-23 16:58:41.798246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 16:58:42.241920: W tensorflow/compiler/tf2t

In [2]:
# Be sure that openFOam is in the path, for instance by running
# 'source /opt/openfoam10/etc/bashrc' before starting the notebook

# The following statement should return openFoam version
print(f"openFoam version: {FoamInformation.foamVersion()}")

openFoam version: 10


In [2]:
%autoreload 2
mesh_size = f3dasm.DiscreteParameter(
    name="mesh_factor", lower_bound=1, upper_bound=10
)
input_velocity = f3dasm.ContinuousParameter(
    name="input_velocity", lower_bound=0.1, upper_bound=10.0
)
output_pressure = f3dasm.ContinuousParameter(
    name="output_pressure", lower_bound=0.0, upper_bound=1.0
)

avg_error = f3dasm.ContinuousParameter(name="avg_error")

design = f3dasm.DesignSpace(
    input_space=[mesh_size, input_velocity, output_pressure],
    output_space=[avg_error],
)

# Create the sampler object
sampler = f3dasm.sampling.RandomUniform(design=design, seed=42)

data: f3dasm.ExperimentData = sampler.get_samples(numsamples=5)

# f3dasm.experiment.quickstart()

job_queue = f3dasm.experiment.JobQueue.from_experimentdata(
    filename="my_jobs", experimentdata=data
)

job_queue.write_new_jobfile()#overwrite=True)
data.store('data')

In [3]:
%autoreload 2

def construct_parameters(names:list, args: dict):
    parameters = {}
    for name in names:
        parameters[name]=dict(value=args[name], description=name)

    return parameters

while True:
    try:
        job_id = job_queue.get()
       
    except f3dasm.experiment.NoOpenJobsError:
        break

    # data = f3dasm.design.("data")
 
    args = data.get_inputdata_by_index(job_id)

  

    # Create simulation info
    material=openFoam.material.Material()
    geometry=openFoam.geometry.Geometry()
    boundary=openFoam.boundary.Boundary(parameters=construct_parameters(names=["input_velocity", "output_pressure"], args=args))
    mesh=openFoam.mesh.Mesh(parameters=construct_parameters(names=["mesh_factor"], args=args))
    solver=openFoam.solver.Solver()
    simulation_info = openFoam.simulator_info.SimulationInfo(material=material, geometry=geometry, boundary=boundary, mesh=mesh, solver=solver,
                                                             case_source_path=Path("cylinder.orig"),
        job_id=job_id)

    simulator_info = openFoam.simulator_info.SimulatorInfo(fork="", version="", build_date="")

    simulator = openFoam.openfoam_simulator.openFoamSimulator(simulation_info=simulation_info, simulator_info=simulator_info)

    # print(simulator.simulation_info.to_dict())

    simulator.pre_process(overwrite=True)

    simulator.execute(solver_options=["-withFunctionObjects", "-writePhi", "-writep"])

    simulator.post_process()

    value = simulator.results["avg_error"]
    data.set_outputdata_by_index(job_id, value)

    data.store("data")

    job_queue.mark_finished(job_id)


data.data

 PyFoam WARNING on line 698 of file /home/guillaume/miniconda3/envs/f3dasm_env/lib/python3.10/site-packages/PyFoam/RunDictionary/SolutionDirectory.py : The first timestep in /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0  is  None not a number. Doing nothing 
2023-06-23 16:58:46,193 - Starting: blockMesh -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0 in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:58:46,212 - Started with PID 76889


 Warning in /home/guillaume/miniconda3/envs/f3dasm_env/bin/pyFoamPrepareCase.py : Case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0 does not have a 'constant' directory 
Using default values from /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0/default.parameters
Looking for template values /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0

Used values

              Name - Value
----------------------------------------
          caseName - "cylinder_0"
          casePath - "/home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0"
          foamFork - openfoam
       foamVersion - 10
    input_velocity - 3.807947176588889
       mesh_factor - 6.0
numberOfProcessors - 1
   output_pressure - 0.9507143064099162

No script /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0/derivedParameters.py for derived values
Clearing /home/guillaume/Documents/code/f3dasm_simulate/notebooks/

2023-06-23 16:58:46,856 - Finished
2023-06-23 16:58:46,874 - Starting: potentialFoam -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0 -withFunctionObjects -writePhi -writep in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:58:46,888 - Started with PID 77088
2023-06-23 16:58:52,722 - Finished
2023-06-23 16:58:52,724 - Starting: postProcess -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_0 -func streamFunction in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:58:52,741 - Started with PID 77288
2023-06-23 16:58:52,795 - Finished
 PyFoam WARNING on line 698 of file /home/guillaume/miniconda3/envs/f3dasm_env/lib/python3.10/site-packages/PyFoam/RunDictionary/SolutionDirectory.py : The first timestep in /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_1  is  None not a number. Doing nothing 
2023-06-23 16:58:52,945 - Starting: blockMesh -case /home/guillaume/Documen

 Warning in /home/guillaume/miniconda3/envs/f3dasm_env/bin/pyFoamPrepareCase.py : Case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_1 does not have a 'constant' directory 
Using default values from /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_1/default.parameters
Looking for template values /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_1

Used values

              Name - Value
----------------------------------------
          caseName - "cylinder_1"
          casePath - "/home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_1"
          foamFork - openfoam
       foamVersion - 10
    input_velocity - 7.34674002393291
       mesh_factor - 5.0
numberOfProcessors - 1
   output_pressure - 0.5986584841970366

No script /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_1/derivedParameters.py for derived values
Clearing /home/guillaume/Documents/code/f3dasm_simulate/notebooks/j

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/guillaume/.local/lib/python3.10/site-packages/psutil/_common.py", line 443, in wrapper
    ret = self._cache[fun]
AttributeError: 'Process' object has no attribute '_cache'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/guillaume/.local/lib/python3.10/site-packages/psutil/_pslinux.py", line 1645, in wrapper
    return fun(self, *args, **kwargs)
  File "/home/guillaume/.local/lib/python3.10/site-packages/psutil/_common.py", line 446, in wrapper
    return fun(self)
  File "/home/guillaume/.local/lib/python3.10/site-packages/psutil/_pslinux.py", line 1687, in _parse_stat_file
    data = bcat("%s/%s/stat" % (self._procfs_path, self.pid))
  File "/home/guillaume/.local/lib/python3.10/site-packages/psutil/_common.py", line 776, in bcat
    return cat(fname, fallback=fallback, _open=open_binary)
  File "/home/guillaume/.local/lib/python3.10/s

...

 Warning in /home/guillaume/miniconda3/envs/f3dasm_env/bin/pyFoamPrepareCase.py : Case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3 does not have a 'constant' directory 
Using default values from /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3/default.parameters
Looking for template values /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3

Used values

              Name - Value
----------------------------------------
          caseName - "cylinder_3"
          casePath - "/home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3"
          foamFork - openfoam
       foamVersion - 10
    input_velocity - 0.6750277604651747
       mesh_factor - 8.0
numberOfProcessors - 1
   output_pressure - 0.8661761457749352

No script /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3/derivedParameters.py for derived values
Clearing /home/guillaume/Documents/code/f3dasm_simulate/notebooks

2023-06-23 16:59:08,006 - Finished
2023-06-23 16:59:08,026 - Starting: potentialFoam -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3 -withFunctionObjects -writePhi -writep in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:59:08,043 - Started with PID 78433
2023-06-23 16:59:14,821 - Finished
2023-06-23 16:59:14,823 - Starting: postProcess -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_3 -func streamFunction in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:59:14,842 - Started with PID 78640
2023-06-23 16:59:14,916 - Finished
 PyFoam WARNING on line 698 of file /home/guillaume/miniconda3/envs/f3dasm_env/lib/python3.10/site-packages/PyFoam/RunDictionary/SolutionDirectory.py : The first timestep in /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4  is  None not a number. Doing nothing 
2023-06-23 16:59:15,106 - Starting: blockMesh -case /home/guillaume/Documen

 Warning in /home/guillaume/miniconda3/envs/f3dasm_env/bin/pyFoamPrepareCase.py : Case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4 does not have a 'constant' directory 
Using default values from /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4/default.parameters
Looking for template values /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4

Used values

              Name - Value
----------------------------------------
          caseName - "cylinder_4"
          casePath - "/home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4"
          foamFork - openfoam
       foamVersion - 10
    input_velocity - 6.051038616257767
       mesh_factor - 6.0
numberOfProcessors - 1
   output_pressure - 0.7080725777960455

No script /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4/derivedParameters.py for derived values
Clearing /home/guillaume/Documents/code/f3dasm_simulate/notebooks/

2023-06-23 16:59:15,827 - Finished
2023-06-23 16:59:15,852 - Starting: potentialFoam -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4 -withFunctionObjects -writePhi -writep in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:59:15,869 - Started with PID 78879
2023-06-23 16:59:22,914 - Finished
2023-06-23 16:59:22,915 - Starting: postProcess -case /home/guillaume/Documents/code/f3dasm_simulate/notebooks/jobs/cylinder_4 -func streamFunction in /home/guillaume/Documents/code/f3dasm_simulate/notebooks
2023-06-23 16:59:22,934 - Started with PID 79093
2023-06-23 16:59:22,998 - Finished
2023-06-23 16:59:23,026 - An unexpected error occurred: The jobfile my_jobs does not have any open jobs left!


input                                   output
  mesh_factor input_velocity output_pressure avg_error
0           6       3.807947        0.950714  0.053566
1           5       7.346740        0.598658  0.053693
2           2       1.644585        0.155995  0.054690
3           8       0.675028        0.866176  0.053637
4           6       6.051039        0.708073  0.053566

In [12]:
simulator.solution_dir.writeDictionaryContents(
    directory="templates", name="parameters.template", contents=simulator.simulation_info.get_structured_parameters()
)

In [5]:
%autoreload 2
simulator.simulation_info.get_structured_parameters()

{}

In [3]:
issubclass(openFoam.boundary.Boundary, openFoam.simulator_part.ParametrizedPart)

True

In [29]:
s=simulator.solution_dir.getDictionaryContents(directory="0", name="error")["internalField"]

In [41]:

sum(s.val)/len(s.val)


0.05358638156499998